In [6]:
import os
import sys
import numpy as np
import pandas as pd


In [7]:
train_data = pd.read_csv("train.tsv", sep='\t', header=None)

In [8]:
train_data.columns = ['claimID', 'claim', 'label', 
                      'claimURL', 'reason', 'categories', 
                      'speaker', 'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

In [9]:
train_data.head(1)

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,pomt-03627,Six out of 10 of the highest unemployment rate...,half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None']


In [14]:
pd.isnull(train_data['claimID']).sum()

0

In [15]:
pd.isnull(train_data['claim']).sum()

1

In [17]:
claim = train_data['claim']
label = train_data['label']
reason = train_data['reason']

In [16]:
# training data later 

def create_dataloader_training(features):
    # The next lines are taken from the example at https://github.com/huggingface/transformers/blob/0cb163865a4c761c226b151283309eedb2b1ca4d/transformers/data/processors/glue.py#L30
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

    class_counts = np.bincount(all_labels)
    class_sample_freq = 1 / class_counts
    weights = [class_sample_freq[label] for label in all_labels]
    print(class_counts)
    print(class_sample_freq)
    num_samples = round(class_counts[1] * 2).item()  # .item to convert to native int
    print(f'Num samples: {num_samples}')
    sampler = WeightedRandomSampler(weights, num_samples=num_samples,
                                    replacement=True)  # we want to use all positive instances and use equally as many negative instances. This should now generally happen by chance
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [23]:
len(claim)

27871

In [24]:
claim[2]

'Forward an email for Jasmine'

In [28]:
m = []
for i in (0,3):
    t = tokenizer.tokenize(claim[i])
    m.append(t)
    
print(m)

[['six', 'out', 'of', '10', 'of', 'the', 'highest', 'unemployment', 'rates', 'are', 'also', 'in', 'so', '-', 'called', 'right', 'to', 'work', 'states', '.'], ['pope', 'francis', 'endorsed', 'donald', 'trump', 'for', 'president', '.']]


In [29]:
# trying out BERT tokenizer - making a list of lists for the claim

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
# tokenizes claim from 

tokenized_claim = []

for i in range(len(claim)):
    t = tokenizer.tokenize(claim[i])
    tokenized_claim.append(t)
    

TypeError: 'float' object is not iterable